In [1]:
#!/usr/bin/python3
import pandas as pd
import numpy as np
import re
from ggplot import *
from scipy.stats import gmean
#from stack overflow to set number format
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
first_part = "https://raw.githubusercontent.com/"
second_part = "jlaurito/CUNY_IS608/master/lecture4/data/riverkeeper_data_2013.csv"
try:
    h2o = pd.read_csv(first_part + second_part)
except:
    h2o = pd.read_csv("riverdata.csv")

In [3]:
#take a peek at the data
print(h2o.head())
h2o.dtypes

                        Site        Date EnteroCount  FourDayRainTotal  \
0  Hudson above Mohawk River  10/16/2011        1733             1.500   
1  Hudson above Mohawk River  10/21/2013           4             0.200   
2  Hudson above Mohawk River   9/21/2013          20             0.000   
3  Hudson above Mohawk River   8/19/2013           6             0.000   
4  Hudson above Mohawk River   7/21/2013          31             0.000   

   SampleCount  
0           35  
1           35  
2           35  
3           35  
4           35  


Site                 object
Date                 object
EnteroCount          object
FourDayRainTotal    float64
SampleCount           int64
dtype: object

In [4]:
#change to date-time
h2o['Date'] = pd.to_datetime(h2o['Date'])
print(h2o['Date'].sort_values().head())
print(h2o['Date'].sort_values().tail())

1587   2006-09-19
1752   2006-09-19
1799   2006-09-19
1889   2006-09-19
1661   2006-09-19
Name: Date, dtype: datetime64[ns]
39    2013-10-21
177   2013-10-21
427   2013-10-21
312   2013-10-21
468   2013-10-21
Name: Date, dtype: datetime64[ns]


In [5]:
#remove < > from some of the counts
h2o['EnteroCount'] = h2o['EnteroCount'].replace(">|<", "", regex=True)
h2o['EnteroCount'] = pd.to_numeric(h2o['EnteroCount'], errors="raise") 
h2o.dtypes

Site                        object
Date                datetime64[ns]
EnteroCount                  int64
FourDayRainTotal           float64
SampleCount                  int64
dtype: object

In [6]:
#add a column for good and bad samples
h2o = h2o.assign(bad_result = lambda x: h2o.EnteroCount > 110,
                 good_result = lambda x: h2o.EnteroCount <= 30)
h2o.head()

,Site,Date,EnteroCount,FourDayRainTotal,SampleCount,bad_result,good_result
0,Hudson above Mohawk River,2011-10-16,1733,1.500,35,True,False
1,Hudson above Mohawk River,2013-10-21,4,0.200,35,False,True
2,Hudson above Mohawk River,2013-09-21,20,0.000,35,False,True
3,Hudson above Mohawk River,2013-08-19,6,0.000,35,False,True
4,Hudson above Mohawk River,2013-07-21,31,0.000,35,False,False


In [7]:
#group the data to create a new data frame
h2o_group = h2o.groupby('Site')
h2o_summary = pd.DataFrame(h2o_group.sum())
h2o_summary = h2o_summary.assign(n_tested = h2o_group['EnteroCount'].size(),
                                 site_max = h2o_group['EnteroCount'].max(),
                                 site_min = h2o_group['EnteroCount'].min()).reset_index()
h2o_summary.head()

,Site,EnteroCount,FourDayRainTotal,SampleCount,bad_result,good_result,n_tested,site_max,site_min
0,125th St. Pier,11860,50.900,4356,21.000,28.000,66,1500,8
1,79th St. mid-channel,2313,42.200,2401,4.000,39.000,49,1032,1
2,Albany Rowing Dock,10114,22.700,1296,10.000,11.000,36,2420,3
3,Annesville Creek,3170,18.400,1444,5.000,29.000,38,958,5
4,Athens,7046,20.900,1225,5.000,19.000,35,2420,5


In [8]:
#find the product of the EnteroCount for each site
h2o_x = h2o.assign(g_mean = lambda x: h2o.EnteroCount + 1)
x = pd.DataFrame(h2o_x.groupby('Site')['g_mean'].apply(gmean)).reset_index()
h2o_summary = pd.merge(h2o_summary, x, on='Site')
h2o_summary.head()

,Site,EnteroCount,FourDayRainTotal,SampleCount,bad_result,good_result,n_tested,site_max,site_min,g_mean
0,125th St. Pier,11860,50.900,4356,21.000,28.000,66,1500,8,56.162
1,79th St. mid-channel,2313,42.200,2401,4.000,39.000,49,1032,1,17.490
2,Albany Rowing Dock,10114,22.700,1296,10.000,11.000,36,2420,3,70.950
3,Annesville Creek,3170,18.400,1444,5.000,29.000,38,958,5,23.171
4,Athens,7046,20.900,1225,5.000,19.000,35,2420,5,41.779


In [9]:
#new columns for geometric mean, good/ok/bad sample rates
h2o_summary = h2o_summary.assign(good_rate = h2o_summary.good_result/h2o_summary.n_tested,
                                 bad_rate = h2o_summary.bad_result/h2o_summary.n_tested)
h2o_summary = h2o_summary.assign(ok_rate = 1 - h2o_summary.good_rate - h2o_summary.bad_rate)
h2o_summary.head()

,Site,EnteroCount,FourDayRainTotal,SampleCount,bad_result,good_result,n_tested,site_max,site_min,g_mean,bad_rate,good_rate,ok_rate
0,125th St. Pier,11860,50.900,4356,21.000,28.000,66,1500,8,56.162,0.318,0.424,0.258
1,79th St. mid-channel,2313,42.200,2401,4.000,39.000,49,1032,1,17.490,0.082,0.796,0.122
2,Albany Rowing Dock,10114,22.700,1296,10.000,11.000,36,2420,3,70.950,0.278,0.306,0.417
3,Annesville Creek,3170,18.400,1444,5.000,29.000,38,958,5,23.171,0.132,0.763,0.105
4,Athens,7046,20.900,1225,5.000,19.000,35,2420,5,41.779,0.143,0.543,0.314


In [10]:
#top 10 best places to swim
best = h2o_summary.sort_values('g_mean').head(10).reset_index()
best.loc[:,['Site','g_mean','good_rate']]

,Site,g_mean,good_rate
0,Norrie Point mid-channel,5.649,0.917
1,Poughkeepsie Drinking Water Intake,5.877,0.974
2,Port Ewen Drinking Water Intake,7.237,0.892
3,Tivoli Landing,9.109,0.838
4,Little Stony Point,9.739,0.895
5,Ulster Landing Beach,11.263,0.833
6,West Point STP Outfall,11.310,0.833
7,TZ Bridge mid-channel,11.509,0.912
8,Marlboro Landing,11.608,0.829
9,Kingston Point Beach,11.814,0.767


In [11]:
#worst 10 places to swim
worst = h2o_summary.sort_values('g_mean', ascending=False).head(10).reset_index()
worst.loc[:,['Site','g_mean','bad_rate']]

,Site,g_mean,bad_rate
0,Upper Sparkill Creek,391.959,0.794
1,Gowanus Canal,187.808,0.459
2,Mohawk River at Waterford,173.834,0.571
3,Newtown Creek- Metropolitan Ave. Bridge,153.318,0.491
4,Saw Mill River,118.737,0.440
5,Piermont Pier,110.256,0.492
6,Hudson River above Troy Lock,107.786,0.351
7,Newburgh Launch Ramp,107.682,0.526
8,Kingston STP Outfall,105.750,0.350
9,Dunn Memorial Bridge- Albany,100.649,0.395


In [33]:
p = ggplot(best, aes(x='Site', y='g_mean'))
p = p + geom_bar(stat='identity')
p = p + ggtitle('Best Spots to Take a Dip')
p

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


<ggplot: (-9223372036579584824)>

In [34]:
p = ggplot(worst, aes(x='Site', y='g_mean'))
p = p + geom_bar(stat='identity')
p = p + ggtitle('Worst Spots to Take a Dip')
p

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


<ggplot: (273092693)>

In [13]:
#most tested
most = h2o_summary.sort_values('n_tested', ascending=False).head(10).reset_index()
most.loc[:,['Site','n_tested']]

,Site,n_tested
0,Piermont Pier,187
1,Upper Sparkill Creek,165
2,125th St. Pier,66
3,Nyack Launch Ramp,61
4,Newtown Creek- Dutch Kills,57
5,TZ Bridge mid-channel,57
6,Orangetown STP Outfall,57
7,Newtown Creek- Metropolitan Ave. Bridge,57
8,Yonkers mid-channel,52
9,Yonkers STP Outfall,51


In [14]:
#find gaps
gaps = h2o.sort_values(['Site','Date']).reset_index(drop=True)
#find the gap between each date
#dividing by np.timedelta yields numeric results 
gaps = gaps.assign(gap = lambda x: (gaps['Date'] - gaps['Date'].shift())/np.timedelta64(1, 'D'))
#but if the site changes the gap resets
gaps['gap'] = np.where(gaps['Site'] != gaps['Site'].shift(), 0, gaps['gap'])
gaps['gaps_csum'] = gaps.groupby('Site')['gap'].cumsum()
gaps.loc[64:68,]

,Site,Date,EnteroCount,FourDayRainTotal,SampleCount,bad_result,good_result,gap,gaps_csum
64,125th St. Pier,2013-09-18,41,0.000,66,False,False,35.000,2543.000
65,125th St. Pier,2013-10-16,201,0.000,66,True,False,28.000,2571.000
66,79th St. mid-channel,2006-09-26,10,0.100,49,False,True,0.000,0.000
67,79th St. mid-channel,2006-10-18,30,1.100,49,False,True,22.000,22.000
68,79th St. mid-channel,2006-11-10,143,3.600,49,True,False,23.000,45.000


In [15]:
#top 20 gaps
pd.set_option('display.float_format', lambda x: '%.0f' % x)
gaps.sort_values('gap', ascending=False).head(20).reset_index().loc[:,['Site','gap','Date']]

,Site,gap,Date
0,Tarrytown Marina,334,2010-07-22
1,Tarrytown Marina,329,2009-08-22
2,125th St. Pier,291,2008-05-19
3,Gowanus Canal,281,2010-06-10
4,Upper Sparkill Creek,280,2008-05-12
5,Newtown Creek- Metropolitan Ave. Bridge,256,2008-05-19
6,East River mid-channel at Roosevelt Is.,256,2008-05-19
7,Yonkers mid-channel,256,2008-05-19
8,The Battery mid-channel,256,2008-05-19
9,Newtown Creek- Dutch Kills,256,2008-05-19


In [16]:
#take a look at when the most tested sites got tested
most_tested = gaps.assign(most=gaps['Site'].isin(most['Site']))
most_tested = most_tested.loc[most_tested['most'] == True]
p = ggplot(most_tested, aes(x='gaps_csum', y=1, color='Site')) + geom_point(size=5)
p = p + facet_grid('Site')
p

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


<ggplot: (-9223372036582010494)>

In [17]:
#rain water and quailty
h2o['EnteroCount'].corr(h2o['FourDayRainTotal'])

0.14482598724767229

In [18]:
#graph
p = ggplot(h2o.loc[h2o['EnteroCount'] < 2400], aes(x='FourDayRainTotal', y='EnteroCount', color='Site'))
p = p + geom_point()
p

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


<ggplot: (281051098)>

In [19]:
#check each site for correlations
pd.set_option('display.float_format', lambda x: '%.3f' % x)
h2o_cor = h2o.groupby('Site')[['FourDayRainTotal','EnteroCount']].corr()
h2o_cor = pd.DataFrame(h2o_cor.ix[0::2,'EnteroCount'].sort_values(ascending=False)).reset_index()
h2o_cor = h2o_cor.drop(['level_1'], axis=1)
h2o_cor.columns=['Site','cor']
h2o_cor = h2o_cor.assign(r_2 = h2o_cor['cor']*h2o_cor['cor'])
high_cor = h2o_cor.head(10)
high_cor

,Site,cor,r_2
0,Gay's Point mid-channel,0.729,0.532
1,Norrie Point Yacht Basin,0.719,0.517
2,The Battery mid-channel,0.699,0.489
3,Esopus Creek Entrance,0.693,0.481
4,Wappingers Creek,0.688,0.473
5,TZ Bridge mid-channel,0.676,0.457
6,Esopus Creek West,0.654,0.428
7,Poughkeepsie Launch Ramp,0.633,0.400
8,East River mid-channel at Roosevelt Is.,0.628,0.394
9,Athens,0.621,0.385
